# Week 13
# GroupBy Mechanics

Many data processing follows a **split-apply-combine** process. For example, you may want to do the following operations to analyze a dataset about sales:
1. What is the total revenue of each day?
2. What is the total sales of each product?
3. How much has each client perchased in total?

These operations all requires that you split the data into groups, and then apply certain calculations to each of the groups, and finally combine all results into a new table. In Pandas this is mostly done with `groupby()` function.

**Readings**
- Textbook, Chapter 8

In [1]:
import numpy as np
import pandas as pd

In [2]:
# An example:
df = pd.DataFrame({'Name' : ['Alice', 'Bob', 'Charlie', 'Alice', 'Bob', 'Charlie'],
                   'Course' : ['Programming', 'Programming', 'Programming','Data Structure', 'Data Structure','Data Structure',],
                   'Semester': ['Spring 2019', 'Fall 2019', 'Fall 2019', 'Spring 2019', 'Fall 2019', 'Spring 2019'],
                   'Homework' : np.random.randint(60, 100, size=6),
                   'Exam' : np.random.randint(60, 100, size=6)})
df

,Name,Course,Semester,Homework,Exam
0,Alice,Programming,Spring 2019,81,83
1,Bob,Programming,Fall 2019,90,98
2,Charlie,Programming,Fall 2019,65,86
3,Alice,Data Structure,Spring 2019,97,63
4,Bob,Data Structure,Fall 2019,78,64
5,Charlie,Data Structure,Spring 2019,79,93


In [5]:
# Split exam scores according to name
groups = df['Exam'].groupby(df['Name'])

groups

In [6]:
# Apply mean() function to find the average value for each group
means = groups.mean()

means

Name
Alice      73.0
Bob        81.0
Charlie    89.5
Name: Exam, dtype: float64

We obtain a **data series**. It can be converted to a data frame by `to_frame()` method.

In [7]:
# A common practice is to convert the results to a data frame
df_means = means.to_frame(name='Average Exam Score')

df_means

,Average Exam Score
Name,
Alice,73.0
Bob,81.0
Charlie,89.5


In [8]:
# Put all operations in one statement
df_means = df['Exam'].groupby(df['Name']).mean().to_frame(name = 'Average Exam Score')

df_means

,Average Exam Score
Name,
Alice,73.0
Bob,81.0
Charlie,89.5


In [26]:
# Exercise: Find the average homework score and exam score for each course
# 1 let find avg of Home Work Score
groups = df['Homework'].groupby(df['Course'])
means = groups.mean()
df_average_home_work = means.to_frame(name='Average Home Work')
df_average_home_work

# 2 let find avg of Exam Score for each course
groups = df['Exam'].groupby(df['Course'])
mean_exam = groups.mean()
df_avg_exam = mean_exam.to_frame(name='Average Exam Score')
df_avg_exam

# let merge df avg home work and df avg exame score
#pd.merge(df_average_home_work, df_avg_exam, on='Course') # merge course attribute
pd.merge(df_average_home_work, df_avg_exam, left_index = True, right_index = True) # merge on index attribute

,Average Home Work,Average Exam Score
Course,,
Data Structure,84.666667,73.333333
Programming,78.666667,89.000000


## Split Data with Multiple Columns

We can use more than one column as keys to split data into groups.

In [32]:
# Split the exam score according to both course name and semester.
# for grouping start always by values endthen keys
groups = df['Exam'].groupby([df['Course'], df['Semester']])

In [33]:
# Calculate the average score
means = groups.mean()

means

Course          Semester   
Data Structure  Fall 2019      64
                Spring 2019    78
Programming     Fall 2019      92
                Spring 2019    83
Name: Exam, dtype: int32

In [34]:
# Convert the result to a data frame
df_means = means.to_frame(name='Average Exam Score')

df_means

Average Exam Score
Course         Semester                       
Data Structure Fall 2019                    64
               Spring 2019                  78
Programming    Fall 2019                    92
               Spring 2019                  83

`means` is a data series with **hierarchical indexing**. It can be converted to a data frame using `unstack()`.

In [35]:
means.index

MultiIndex([('Data Structure',   'Fall 2019'),
            ('Data Structure', 'Spring 2019'),
            (   'Programming',   'Fall 2019'),
            (   'Programming', 'Spring 2019')],
           names=['Course', 'Semester'])

In [37]:
means.unstack() # unstack convert data series with multiple indices in into a data frame 

Semester,Fall 2019,Spring 2019
Course,,
Data Structure,64,78
Programming,92,83


We can specify which index to be unstacked.

In [38]:
means.unstack(level=0)

Course,Data Structure,Programming
Semester,,
Fall 2019,64,92
Spring 2019,78,83


In [39]:
means.unstack(level=1)

Semester,Fall 2019,Spring 2019
Course,,
Data Structure,64,78
Programming,92,83


In [44]:
# Exercise:
# Using one statement, create the above data frame directly from df.
# always three steps split combine frame
df['Exam'].groupby([df['Course'], df['Semester']]).mean().unstack()

Semester,Fall 2019,Spring 2019
Course,,
Data Structure,64,78
Programming,92,83


We can split the entire data set instead of one column.

In [45]:
df.groupby([df['Course'], df['Semester']]).mean()

Homework  Exam
Course         Semester                   
Data Structure Fall 2019        78.0  64.0
               Spring 2019      88.0  78.0
Programming    Fall 2019        77.5  92.0
               Spring 2019      81.0  83.0

In [46]:
# Frequently the grouping information is found in the same data frame as the data 
# you want to work on. In that case, simply put column names as the keys:
df.groupby(['Course', 'Semester']).mean()

Homework  Exam
Course         Semester                   
Data Structure Fall 2019        78.0  64.0
               Spring 2019      88.0  78.0
Programming    Fall 2019        77.5  92.0
               Spring 2019      81.0  83.0

In [49]:
# Exercise:
# Use `size()` method to find the number of students for each course 
# in each semester
df.groupby(['Course', 'Semester']).size().to_frame(name = "Number of Students")

Number of Students
Course         Semester                       
Data Structure Fall 2019                     1
               Spring 2019                   2
Programming    Fall 2019                     2
               Spring 2019                   1

## Iterating Over Groups

The GroupBy object support iteration, providing a sequence of 2-tuples containing the group name along with the data.

In [50]:
# Show the content of each group.
groups = df.groupby('Name')
for name, group in groups:
    print("Name:", name)
    print(group)

Name: Alice
    Name          Course     Semester  Homework  Exam
0  Alice     Programming  Spring 2019        81    83
3  Alice  Data Structure  Spring 2019        97    63
Name: Bob
  Name          Course   Semester  Homework  Exam
1  Bob     Programming  Fall 2019        90    98
4  Bob  Data Structure  Fall 2019        78    64
Name: Charlie
      Name          Course     Semester  Homework  Exam
2  Charlie     Programming    Fall 2019        65    86
5  Charlie  Data Structure  Spring 2019        79    93


**Syntactic sugar**: It is simpler to use the following statement for selecting columns for groupby()

In [52]:
df.groupby('Name')['Exam'].mean()
# privesiously we use df['Exam'].groupby(df['Name'])

Name
Alice      73.0
Bob        81.0
Charlie    89.5
Name: Exam, dtype: float64

In [53]:
# The standard statement
df['Exam'].groupby(df['Name']).mean()

Name
Alice      73.0
Bob        81.0
Charlie    89.5
Name: Exam, dtype: float64

In [54]:
# The following statement does not work because 
# 'Name' is not a column in df['Exam']

# df['Exam'].groupby('Name').mean()

In [60]:
df.groupby('Semester')['Homework'].mean().to_frame(name = 'avg')

,avg
Semester,
Fall 2019,77.666667
Spring 2019,85.666667


## **Grouping with dictionary**
We can use a seperate dictionary to decide the groups.

In [61]:
genders = {
    "Alice": "Female",
    "Bob": "Male",
    "Charlie": "Male"
}

In [56]:
data = df.set_index("Name")

data

,Course,Semester,Homework,Exam
Name,,,,
Alice,Programming,Spring 2019,81,83
Bob,Programming,Fall 2019,90,98
Charlie,Programming,Fall 2019,65,86
Alice,Data Structure,Spring 2019,97,63
Bob,Data Structure,Fall 2019,78,64
Charlie,Data Structure,Spring 2019,79,93


In [62]:
# Split the data according to gender
data.groupby(genders).size()

Female    2
Male      4
dtype: int64

In [63]:
# One can also use a list to indicate the grouping information
# Not recommended because it is hard to understand
genders = ['F', 'M', 'M', 'F', 'M', 'M']

data.groupby(genders).size()

F    2
M    4
dtype: int64

In [64]:
# Exercise: Calculate the average scores for each gender
data.groupby(genders).mean()

,Homework,Exam
F,89.0,73.00
M,78.0,85.25


## Grouping with functions

Any function passed as a group key will be called once per index value, with the returned values being used as the group names.

In [65]:
def get_initial(name):
    return name[0]

In [67]:
get_initial('Falilou')

'F'

In [68]:
data.groupby(get_initial).mean()

,Homework,Exam
A,89.0,73.0
B,84.0,81.0
C,72.0,89.5


In [69]:
# The function can be defined via lambda expression
data.groupby(lambda x: x[0]).mean()

,Homework,Exam
A,89.0,73.0
B,84.0,81.0
C,72.0,89.5


### Exercise:
Last week's homework asks us to create a data frame with number of airport for each country. Let's think about how it can be achieved using the groupby mechanism.

In [27]:
airports = pd.read_csv('airports.csv')
airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [28]:
countries1_csv = pd.read_csv('countries1.csv')
countries1_csv.head()

,id,code,name,continent,wikipedia_link,keywords
0,302672,AD,Andorra,EU,https://en.wikipedia.org/wiki/Andorra,NaN
1,302618,AE,United Arab Emirates,AS,https://en.wikipedia.org/wiki/United_Arab_Emir...,"UAE,مطارات في الإمارات العربية المتحدة"
2,302619,AF,Afghanistan,AS,https://en.wikipedia.org/wiki/Afghanistan,NaN
3,302722,AG,Antigua and Barbuda,NaN,https://en.wikipedia.org/wiki/Antigua_and_Barbuda,NaN
4,302723,AI,Anguilla,NaN,https://en.wikipedia.org/wiki/Anguilla,NaN


In [29]:
groups =  airports.groupby('iso_country')
results = groups.size()
df = results.to_frame(name = 'Number')
df.head()

,Number
iso_country,
AD,2
AE,53
AF,65
AG,4
AI,2


In [31]:
# add coulmn of the country name
df = pd.merge(df, countries1_csv[['code', 'name']], left_index = True, right_on = 'code')
df.head()

,Number,code,name
0,2,AD,Andorra
1,53,AE,United Arab Emirates
2,65,AF,Afghanistan
3,4,AG,Antigua and Barbuda
4,2,AI,Anguilla
